In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


# IMPORTING LIBRARIES

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import spacy
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
df= pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
sample = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')


In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df.head()
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


# TEXT PRE PROCESSING

In [6]:

stop_words = set(stopwords.words('english'))

In [7]:
cleaned_text_list = []
for t in df['text']:
    # Remove URLs and special characters (keep only alphabets and whitespace)
    t = re.sub(r'http\S+|www\S+|[^a-zA-Z\s]', '', t)
    tokens = word_tokenize(t)
    t= t.lower()
    filtered_tokens = [token for token in tokens if token not in stop_words]
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    cleaned_text_list.append(' '.join(stemmed_tokens))

In [8]:
df['cleaned_text']= cleaned_text_list
df

,id,keyword,location,text,target,cleaned_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deed reason earthquak may allah forgiv us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near La rong sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all resid ask shelter place notifi offic No ev...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,peopl receiv wildfir evacu order california
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent photo rubi alaska smoke wildfir ...
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant crane hold bridg collaps nearbi home
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,ariaahrari thetawniest the control wild fire c...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,M utckm S volcano hawaii
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,polic investig ebik collid car littl portug eb...


In [9]:
df['cleaned_text']

0           our deed reason earthquak may allah forgiv us
1                    forest fire near La rong sask canada
2       all resid ask shelter place notifi offic No ev...
3             peopl receiv wildfir evacu order california
4       just got sent photo rubi alaska smoke wildfir ...
                              ...                        
7608       two giant crane hold bridg collaps nearbi home
7609    ariaahrari thetawniest the control wild fire c...
7610                             M utckm S volcano hawaii
7611    polic investig ebik collid car littl portug eb...
7612    the latest more home raze northern california ...
Name: cleaned_text, Length: 7613, dtype: object

In [10]:
df_cleaned = df['cleaned_text'].copy()
df_cleaned

0           our deed reason earthquak may allah forgiv us
1                    forest fire near La rong sask canada
2       all resid ask shelter place notifi offic No ev...
3             peopl receiv wildfir evacu order california
4       just got sent photo rubi alaska smoke wildfir ...
                              ...                        
7608       two giant crane hold bridg collaps nearbi home
7609    ariaahrari thetawniest the control wild fire c...
7610                             M utckm S volcano hawaii
7611    polic investig ebik collid car littl portug eb...
7612    the latest more home raze northern california ...
Name: cleaned_text, Length: 7613, dtype: object

In [11]:
cleaned_test_list = []
for a in test['text']:
    # Remove URLs and special characters (keep only alphabets and whitespace)
    a = re.sub(r'http\S+|www\S+|[^a-zA-Z\s]', '', a)
    tokens = word_tokenize(a)
    a= a.lower()
    filtered_tokens = [token for token in tokens if token not in stop_words]
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    cleaned_test_list.append(' '.join(stemmed_tokens))

In [12]:
test['cleaned_test']=cleaned_test_list
test_x=test['cleaned_test']
test_x


0                           just happen terribl car crash
1           heard earthquak differ citi stay safe everyon
2       forest fire spot pond gees flee across street ...
3                          apocalyps light spokan wildfir
4                      typhoon soudelor kill china taiwan
                              ...                        
3258         earthquak safeti lo angel safeti fasten xrwn
3259    storm RI wors last hurrican My cityampoth hard...
3260                            green line derail chicago
3261                  meg issu hazard weather outlook hwo
3262      cityofcalgari activ municip emerg plan yycstorm
Name: cleaned_test, Length: 3263, dtype: object

# FEATURE EXTRACTION


In [13]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_features = tfidf_vectorizer.fit_transform(df_cleaned)
tfidf_testing=tfidf_vectorizer.transform(test_x)

In [14]:
print(tfidf_features.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
x = df['cleaned_text']
y = df['target']

In [16]:
x_train, x_test,y_train , y_test = train_test_split(tfidf_features ,y , test_size=0.2, random_state=42)

# ML ALGORITHM

In [17]:
svm_model = SVC(kernel='linear', C=1.0, random_state=42)

svm_model.fit(x_train, y_train)

SVC(kernel='linear', random_state=42)

In [18]:
y_pred = svm_model.predict(x_test)
test_p=svm_model.predict(tfidf_testing)

In [19]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[763 111]
 [209 440]]


In [20]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.79


In [21]:
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.87      0.83       874
           1       0.80      0.68      0.73       649

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523



In [22]:
sample

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [23]:
sample['target'] = test_p
sample.to_csv('submission.csv', index=False)

In [24]:
sample


,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,0
3260,10868,1
3261,10874,1
